### Configuration

In [1]:
# %pip install bs4
# %pip install requests
# %pip install openpyxl

### Importing librarys

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urlencode

### Setting variables and functions

In [3]:
HEADERS = ({'User-Agent':
			'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
			AppleWebKit/537.36 (KHTML, like Gecko) \
			Chrome/90.0.4430.212 Safari/537.36',
			'Accept-Language': 'en-US, en;q=0.5'})

# user define function
# Scrape the data
def getdata(url):
	r = requests.get(url, headers=HEADERS)
	return r.text


def html_code(url):
	# pass the url
	# into getdata function
	htmldata = getdata(url)
	soup = BeautifulSoup(htmldata, 'html.parser')

	# display html code
	return (soup)

# Info da pessoa

def cus_data(soup):
    # find the Html tag
    # with find()
    # and convert into string
    data_str = ""
    cus_list = []
  
    for item in soup.find_all("span", class_="a-profile-name"):
        data_str = data_str + item.get_text()
        cus_list.append(data_str)
        data_str = ""
    del cus_list[0]
    del cus_list[0]
    return cus_list

# Review da pessoa

def cus_rev(soup):
    # find the Html tag
    # with find()
    # and convert into string
    data_str = ""
  
    for item in soup.find_all("span", class_="a-size-base review-text review-text-content"):
        data_str = data_str + item.get_text()
  
    result = data_str.split("\n")
    return (result)

# Data do review

def cus_date(soup):
    # find the Html tag
    # with find()
    # and convert into string
    data_str = ""
    cus_dates = []
  
    for item in soup.find_all("span", class_="a-size-base a-color-secondary review-date"):
        data_str = data_str + item.get_text()
        cus_dates.append(data_str)
        data_str = ""
    del cus_dates[0]
    del cus_dates[0]
    return cus_dates

# Info do comentário (rate, titulo, verificacao)

def cus_info(soup):
    # find the Html tag
    # with find()
    # and convert into string
    data_str = ""
    cus_info = []
  
    for item in soup.find_all("a", class_="a-link-normal"):
        data_str = data_str + item.get_text()
        cus_info.append(data_str)
        data_str = ""
    return cus_info


### List of Reviews Pages

In [4]:
url_template = 'https://www.amazon.com.br/Fire-TV-Stick-Streaming/product-reviews/B08C1K6LB2/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber='
urls = []
for i in range(1,31):
  urls.append(url_template+str(i))

# Data de acesso 03/12/2022
# Região de acesso 

### Getting metadata

In [5]:
# url_produto = "https://www.amazon.com.br/Fire-TV-Stick-Streaming/dp/B08C1K6LB2?pd_rd_w=enHGE&content-id=amzn1.sym.ccd42964-5498-4dd6-ba8b-06c2e8dba67a&pf_rd_p=ccd42964-5498-4dd6-ba8b-06c2e8dba67a&pf_rd_r=G9T7RD41H1ZGXWEGCDN8&pd_rd_wg=8SzpO&pd_rd_r=137ad4bc-da2b-45f1-b918-5815b2233488&pd_rd_i=B08C1K6LB2&psc=1&ref_=pd_bap_d_grid_rp_0_1_i"
cus_res = []
rev_data = []
rev_result = []
rev_date = []
acces_date = '03 de dezembro de 2022'
acces_reg = 'Brasil'
acces_link = []
cus_rates = []
cus_rates_num = []
cus_titles = []
cus_checks = []

for url in urls:

  soup = html_code(url)
  rev_data = rev_data + cus_rev(soup)
  cus_res = cus_res + cus_data(soup)
  rev_date = rev_date + cus_date(soup)
  cus_infos = cus_info(soup)

  cus_infos = cus_infos[25:65]
  rates = list(range(0,40,4))
  titles = list(range(1,40,4))
  checks = list(range(2,40,4))

  rtc = zip(rates, titles, checks)

  for r, t, c in rtc:
    
    cus_rates.append(cus_infos[r])
    cus_rates_num.append(float(cus_rates[-1].split()[0].replace(',', '.')))

    cus_titles.append(cus_infos[t].strip('\n'))

    cus_checks.append(cus_infos[c])

for i in rev_data:
    if i == "":
        pass
    else:
        rev_result.append(i)
while len(rev_result) < len(cus_res):
  rev_result.append('')

for i in range(len(cus_res)):
  acces_link.append(urls[i//10])

### Checking arrays

In [6]:
print(len(cus_res))
print(len(rev_result))
print(len(rev_date))
print(len(acces_link))
print(len(cus_rates))
print(len(cus_rates_num))
print(len(cus_titles))
print(len(cus_checks))

300
300
300
300
300
300
300
300


### Creating dataset to store metadata

In [9]:
metadados = pd.DataFrame({
  'Nome': cus_res,
  'Rating': cus_rates,
  'Num Rating': cus_rates_num,
  'Título': cus_titles,
  'Comentário': rev_result,
  'Data do comentário': rev_date,
  'Data de acesso': acces_date,
  'Link de acesso': acces_link,
  'Região do comentário': acces_reg,
  'Vericidade da compra': cus_checks
})
metadados.drop_duplicates(inplace=True)
metadados.dropna(inplace=True)

# Removendo comentários nulos
null_comments = metadados['Comentário'] == ''
metadados = metadados[~null_comments]

metadados.reset_index(inplace=True)
metadados.rename(columns={'index': 'id'}, inplace=True)
metadados.to_excel('../data/metadados.xlsx', index=False)